In [3]:
import random, math, copy

## Fonctionnalités de base

In [4]:
# Question 1:
def create_empty_matrix(nbcells, marge=0):
  matrice = []
  for i in range(nbcells + marge * 2):
    submatrice = []
    for j in range(nbcells + marge * 2):
      submatrice.append(0)
    matrice.append(submatrice)
  return matrice


# Question 2:


def xy_to_ij(lst_xy, n_size_mat):
  x, y = lst_xy[0], lst_xy[1]
  # Il est necessaire de differencier le cas >1 et <1, car pour les coordonnees relatives alpha, beta, nous n'appliquons pas la transformation de la meme maniere.
  if y > 1:
    i, j = (-y + (n_size_mat - 1)), x
  else:
    i, j = (-y + 1), x
  return [i, j]

## Affichage de l'état du détecteur

In [6]:
# Question 1:


def print_matrix(matrice):
  for i in range(len(matrice)):
    for j in range(len(matrice[0])):
      print(round(matrice[i][j], 2), end=" ")
    print()


# Question 2:


def print_matrix_bool(matrice):
  for i in range(len(matrice)):
    for j in range(len(matrice[0])):
      if matrice[i][j] == 0:
        print(".", end=" ")
      else:
        print("#", end=" ")
    print()


def print_matrix_energy(matrice):
  for i in range(len(matrice)):
    for j in range(len(matrice[0])):
      if matrice[i][j] > 0.75:
        print("# ", end=" ")
      elif matrice[i][j] > 0.5:
        print("* ", end=" ")
      elif matrice[i][j] > 0.25:
        print("o ", end=" ")
      elif matrice[i][j] > 0.01:
        print(". ", end=" ")
      else:
        print("--", end=" ")
    print()

## Generation d'une particule

In [7]:

# Question 1:
def generate_partic_coords(Xwidth):
  Xvrai = random.random() * Xwidth
  Yvrai = random.random() * Xwidth
  return [round(Xvrai, 2), round(Yvrai, 2)]


# Question 2:


def calc_all_coords(lst_coords_in_det, lgr_cellule):
  colonne = lst_coords_in_det[0] // lgr_cellule
  ligne = lst_coords_in_det[1] // lgr_cellule

  alpha = (lst_coords_in_det[0] % lgr_cellule) / lgr_cellule
  beta = (lst_coords_in_det[1] % lgr_cellule) / lgr_cellule

  return [colonne, ligne], [alpha, beta]

## Simulation du signal créé dans le détecteur

In [8]:

# Fonction donnee (tous les commentaires y sont laisses)
def calculate_fraction_E_in_cells(lst_coords_in_cell, matE, Xwidth, n_cells):
  mat_size = 5
  if len(matE) != mat_size:
    print(
        "\n\n*** ATTENTION ***\nLa taille de la matrice fournie en argument de calculate_fraction_E_in_cells n'a pas l'air correcte.\nSortie anormale de la fonction.\n\n\n"
    )
    return None
  if Xwidth != 100:
    print("\n\n*** ATTENTION ***\nLa largeur de detecteur recue est", Xwidth,
          "cm au lieu de 100 cm.\nSortie anormale de la fonction.\n\n\n")
    return None
  if not (5 <= n_cells <= 50):
    print(
        "\n\n*** ATTENTION ***\nLe nombre", n_cells,
        " de cellules recu est invalide.\nSortie anormale de la fonction.\n\n\n"
    )
    return None
  #Quick trick to see an actual center for large n_cells and still keep fluctuations for low n_cells:
  #nbTirages = int(100 * (n_cells / 20))
  #Let's not use it eventually, fluctuations may be closer to reality this way.
  nbTirages = 100
  #Calo config rather than pixel.
  #In this situation, shower width = 2.5 cm (a-la Moliere radius).
  #In variable sigma though, shower width is stored in units of cell width.
  #Therefore sigma = 2.5cm / cell_width, ie sigma = 2.5cm * n_cells / detector_width.
  #1st limit is when 3 sigmas (in cm) becomes larger than 5x5 matrix diagonal = 2.5 cell_width sqrt(2). This gives n_cells < 2.5 Xwidth sqrt(2) / 3*2.5cm ~ 47.
  #2nd limit is when 3 sigmas (in cm) is all in a single tower, ie n_cells > Xwidth / 3sigma ~ 13.
  sigma = 2.5 * n_cells / Xwidth
  #Just in case the user doesn't provide an empty matrix...
  matRdm = create_empty_matrix(mat_size)
  for i in range(0, nbTirages):
    xRdm = random.gauss(float(mat_size // 2) + lst_coords_in_cell[0], sigma)
    yRdm = random.gauss(float(mat_size // 2) + lst_coords_in_cell[1], sigma)
    #In rare cases the random number is picked outside the interval [0,mat_size[, which triggers an out-of-range error when used in xy_to_ij / matRdm.
    if xRdm < 0:
      xRdm = 0
    if yRdm < 0:
      yRdm = 0
    if xRdm >= mat_size:
      xRdm = mat_size - 0.001
    if yRdm >= mat_size:
      yRdm = mat_size - 0.001
    lst_Rdm = xy_to_ij([int(xRdm), int(yRdm)], mat_size)
    matRdm[lst_Rdm[0]][lst_Rdm[1]] += 1
  for i in range(mat_size):
    for j in range(mat_size):
      matE[i][j] = matRdm[i][j] / nbTirages
  return None


In [9]:
# Question 1:


def smear_energy(matE):
  matE_smeared = copy.deepcopy(matE)
  for i in range(len(matE_smeared)):
    for j in range(len(matE_smeared)):
      matE_smeared[i][j] = round(random.gauss(1, 0.05), 2)
  return matE_smeared


# Question 2:


def apply_threshold(matE_smeared):
  for i in range(len(matE_smeared)):
    for j in range(len(matE_smeared)):
      if matE_smeared[i][j] < 0.05:
        matE_smeared[i][j] = 0


## Assemblage des différentes étapes

In [133]:

def simulate_signal(lst_coords_in_cell, Xwidth, nbcells):
  matE = create_empty_matrix(5, marge=0)
  calculate_fraction_E_in_cells(lst_coords_in_cell, matE, Xwidth, nbcells)
  matE_smeared = smear_energy(
      matE
  )  # Matrice d'energie "distibuee aleatoirement" (copie, donc doit redefinir)
  apply_threshold(matE_smeared)
  return matE_smeared


"""
4.3 Intégration du signal simulé dans la marice globale du détecteur
"""
# Question 1:


def launch_particle_on_detector(Xwidth, nbcells, marge, matrice):  # Attention, la marge n'est pas comprise dans Xwidth!
  lgr_cellule = Xwidth / nbcells
  #Ce sont les coordonnees vis-a-vis du detecteur (Xwidth), la marge n'est pas consideree.
  lst_coords_in_det = generate_partic_coords(Xwidth)
  coord_cellule = calc_all_coords(lst_coords_in_det,
                                  lgr_cellule)[0]  #coordonnees xy
  lst_coords_in_cell = calc_all_coords(lst_coords_in_det,
                                       lgr_cellule)[1]  #coordonnees alpha beta
  coord_cellule_computer = xy_to_ij(coord_cellule, nbcells)
  lst_coords_in_cell_computer = xy_to_ij(lst_coords_in_cell, nbcells)
  matE_smeared = simulate_signal(lst_coords_in_cell, Xwidth, nbcells)
  for i in range(-2, 3):
    for j in range(-2, 3):
      # La matrice a une marge de 2 de chaque cote pour permettre a matE_smeared de rentrer sans erreur d'index
        matrice[int(coord_cellule_computer[0]) + i +
              2][int(coord_cellule_computer[1]) + j +
                 2] = matrice[int(coord_cellule_computer[0]) + i +
                              2][int(coord_cellule_computer[1]) + j +
                                 2] + matE_smeared[i + 2][j + 2]
  print(coord_cellule_computer)
  print("aaaa")
  return lst_coords_in_det


# Question 2:


def create_event(Xwidth, nbcells, marge, matrice, nbparticles):
  liste_coords = []
  # Simule l'arrivee des particules
  for loop in range((nbparticles)):
    launch_particle_on_detector(Xwidth, nbcells, marge, matrice)
    liste_coords.append(launch_particle_on_detector(Xwidth, nbcells, marge, matrice))
  # Enleve la marge de la matrice
  for i in range(len(matrice)):
    for j in range(-2, 2):
      matrice[i].pop(j)
  for i in range(-2, 2):
    matrice.pop(i)
  return liste_coords


# Question 3:


def mon_main(nbcells):
  Xwidth = 100
  marge = 2
  # Demander le nombre d'evenements et particules, verifier qu'il s'agit d'un nombre utilisable (>1).
  nbevents = int(input("Combien d'evenements? "))
  while nbevents < 1:
    nbevents = int(
        input("Saisie erronnee, veuillez recommencer. Combien d'evenements? "))
  nbparticles = int(input("Combien de particules par evenement? "))
  while nbparticles < 1:
    nbparticles = int(
        input(
            "Saisie erronnee, veuillez recommencer. Combien de particules par evenement? "
        ))
  # Creer chaque evenement
  for loop in range(nbevents):
    matrice = create_empty_matrix(nbcells, 2 * marge)
    create_event(Xwidth, nbcells, marge, matrice, nbparticles)
    print_matrix_energy(matrice)
    print('\n')
    print(matrice)
    print_matrix(matrice)


In [136]:
mon_main(10)

Combien d'evenements?  1
Combien de particules par evenement?  3


[4.0, 9.0]
aaaa
[0.0, 5.0]
aaaa
[1.0, 5.0]
aaaa
[1.0, 1.0]
aaaa
[1.0, 9.0]
aaaa
[4.0, 2.0]
aaaa
#  #  #  #  #  #  #  #  #  #  #  #  -- -- 
#  #  #  #  #  #  #  #  #  #  #  #  -- -- 
#  #  #  #  #  #  #  #  #  #  #  #  -- -- 
#  #  #  #  #  #  #  #  #  #  #  #  -- -- 
-- #  #  #  #  -- -- #  #  #  #  #  -- -- 
-- #  #  #  #  -- -- #  #  #  #  #  -- -- 
-- #  #  #  #  -- -- #  #  #  #  #  -- -- 
-- -- -- -- -- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- -- -- -- -- -- 


[[1.07, 1.09, 0.98, 3.09, 2.1100000000000003, 1.93, 2.04, 2.92, 1.03, 0.99, 1.06, 0.99, 0, 0], [0.9, 1.0, 1.05, 3.01, 2.0, 1.94, 1.99, 3.12, 0.99, 0.92, 0.99, 0.97, 0, 0], [1.05, 2.0, 2.04, 3.94, 2.95, 2.0700000000000003, 2.0, 4.220000000000001, 2.0, 2.04, 2.01, 1.76, 0, 0], [1.11, 1.92, 1.970000000

## Reconstruction d'un signal analysable

In [99]:
def mesure_xy(matrice, lgr_cellule, capable_det_E=True):
    

[0.0, 8.0]
[2.0, 4.0]
[6.0, 3.0]
[1.0, 9.0]
[6.0, 0.0]
[7.0, 4.0]
[0.0, 1.0]
[0.0, 3.0]
[1.0, 4.0]
[6.0, 3.0]
#  #  #  #  #  #  #  #  #  #  #  #  -- -- 
#  #  #  #  #  #  #  #  #  #  #  #  -- -- 
#  #  #  #  #  #  #  #  #  #  #  #  -- -- 
-- -- #  #  #  #  #  #  #  #  #  #  -- -- 
#  #  #  #  #  #  #  -- -- -- -- -- -- -- 
#  #  #  #  #  #  #  -- -- -- -- -- -- -- 
#  #  #  #  #  #  #  -- -- -- -- -- -- -- 
#  #  #  #  #  #  #  -- -- -- -- -- -- -- 
#  #  #  #  #  #  #  -- -- -- -- -- -- -- 
-- -- #  #  #  #  #  -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- -- -- -- -- -- 
